<a href="https://colab.research.google.com/github/wenjunc2/PAHO-Drug-Data-Cleaning-Summer-2022/blob/main/2_0_Data_Mapping_for_Chile_1_(Cleaner_Raw_Data%2C_use_scrapy_).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: Set up indexes for clean catagol drug names

In [ ]:
# read into clean catagol name
import pandas as pd
CleanDataBase = pd.read_excel('CleanDataBase.xlsx')
CatalogName = CleanDataBase['Catalog Name']
CatalogName = list(set(CatalogName))
CatalogName = [x for x in CatalogName if str(x) != 'nan']

CatalogNameCopy = CatalogName.copy()
# uppercase everything
CatalogNameU = []
for i in CatalogNameCopy:
  CatalogNameU.append(i.upper())

In [ ]:
CatalogNameU[:5]

['PHENOBARBITAL 50 MG, TABLET, TBD',
 'CARVEDILOL 6.25 MG, TABLET, TBD',
 'FLUCONAZOLE 2 MG/ML, INJECTION, 100 ML VIAL, 10X1',
 'CARBAMAZEPINE 400 MG, TABLET, BLISTER, 10X100',
 'CHLORTHALIDONE 25 MG, TABLET, BLISTER, 10X3']

In [ ]:
# split the CatalogNameU
#from nltk.tokenize import WordPunctTokenizer  # this function split the string by space
#tokenizer = WordPunctTokenizer()

CatalogNameUS = []
for item in CatalogNameU:
  temp = item.split(",")
  CatalogNameUS.append(temp)

In [ ]:
CatalogNameUS[:5] # after split by ","

[['PHENOBARBITAL 50 MG', ' TABLET', ' TBD'],
 ['CARVEDILOL 6.25 MG', ' TABLET', ' TBD'],
 ['FLUCONAZOLE 2 MG/ML', ' INJECTION', ' 100 ML VIAL', ' 10X1'],
 ['CARBAMAZEPINE 400 MG', ' TABLET', ' BLISTER', ' 10X100'],
 ['CHLORTHALIDONE 25 MG', ' TABLET', ' BLISTER', ' 10X3']]

Clean Catalog Drug Name - presentation form

In [ ]:
# summary the list of presentation form
PresentationForm = []

for item in CatalogNameUS:
  if len(item) >= 2:
    PresentationForm.append(item[1])

In [ ]:
PresentationForm = list(set(PresentationForm))

len(PresentationForm)

29

In [ ]:
PresentationForm[:5] # the presentation form

[' SUSPENSION FOR INTRATRACHEAL INSTILLATION',
 ' CAPSULE',
 ' INHALER',
 ' ORAL PELLETS',
 ' 4MG TABLET']

Clean Catalog Drug Name - main medicine : for get index

In [ ]:
# grab the main medicine 
MainMedicine = []

for item in CatalogNameUS:
  MainMedicine.append(item[0])

In [ ]:
print(len(MainMedicine), MainMedicine[:5])

462 ['PHENOBARBITAL 50 MG', 'CARVEDILOL 6.25 MG', 'FLUCONAZOLE 2 MG/ML', 'CARBAMAZEPINE 400 MG', 'CHLORTHALIDONE 25 MG']


In [ ]:
# split MainMedicine to get only the medicne name, and use the medicine name as the index
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

MainMedicineSpl = []
for item in MainMedicine:
  tokenlist = tokenizer.tokenize(item)
  MainMedicineSpl.append(tokenlist)

In [ ]:
MainMedicineSpl[:5]

[['PHENOBARBITAL', '50', 'MG'],
 ['CARVEDILOL', '6', '.', '25', 'MG'],
 ['FLUCONAZOLE', '2', 'MG', '/', 'ML'],
 ['CARBAMAZEPINE', '400', 'MG'],
 ['CHLORTHALIDONE', '25', 'MG']]

In [ ]:
MainMedicineIndex = []
for item in MainMedicineSpl:
  templist = []
  for subitem in item:
    if len(subitem) > 4 and subitem.isdigit() is False:  ## ignoring these elements such as "MG", "+", ... 
      templist.append(subitem)
  MainMedicineIndex.append(templist)


In [ ]:
MainMedicineIndex[:5]

[['PHENOBARBITAL'],
 ['CARVEDILOL'],
 ['FLUCONAZOLE'],
 ['CARBAMAZEPINE'],
 ['CHLORTHALIDONE']]

In [ ]:
CatalogIndexData = {"Catalog Name": CatalogNameU, "Catalog Index": MainMedicineIndex}

In [ ]:
CatalogIndexData_df = pd.DataFrame(CatalogIndexData)

In [ ]:
len(CatalogIndexData_df)

462

In [ ]:
CatalogIndexData_df[:5]

,Catalog Name,Catalog Index
0,"PHENOBARBITAL 50 MG, TABLET, TBD",[PHENOBARBITAL]
1,"CARVEDILOL 6.25 MG, TABLET, TBD",[CARVEDILOL]
2,"FLUCONAZOLE 2 MG/ML, INJECTION, 100 ML VIAL, 10X1",[FLUCONAZOLE]
3,"CARBAMAZEPINE 400 MG, TABLET, BLISTER, 10X100",[CARBAMAZEPINE]
4,"CHLORTHALIDONE 25 MG, TABLET, BLISTER, 10X3",[CHLORTHALIDONE]


List of main medicine which is used to compare and pair with Chile 1

In [ ]:
# create a list for main medicine from catalog name data set
CatalogMedName = []
for item in MainMedicineIndex:
  for subitem in item:
    CatalogMedName.append(subitem)

In [ ]:
CatalogMedName = list(set(CatalogMedName))
len(CatalogMedName)

206

In [ ]:
CatalogMedName[:3]

['RANITIDINE', 'DEXTROMETORFANO', 'MOXIFLOXACIN']

## Part 2: get the chile 1 data set drug names list

In [ ]:
import chardet
with open('CHLDiab1.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'confidence': 0.73, 'encoding': 'Windows-1252', 'language': ''}

In [ ]:
import pandas as pd
CHLDiab1 = pd.read_csv('CHLDiab1.csv',encoding='Windows-1252')
#CHLDiab1.head(1)

In [ ]:
CHLDiab1DirtyName = CHLDiab1['Descripcion linea Adquisicion']

In [ ]:
len(CHLDiab1DirtyName) # length of drug names

20606

In [ ]:
CHLDiab1DirtyName = list(set(CHLDiab1DirtyName))
len(CHLDiab1DirtyName) # length of drug names after remove duplicates

3663

In [ ]:
# uppercase everything
CHLDiab1DirtyNameU = []
for i in CHLDiab1DirtyName:
  CHLDiab1DirtyNameU.append(i.upper())

In [ ]:
CHLDiab1DirtyNameU[:5]

['GLICENEX SR 1000 METFORMINA 1000 MG',
 'INSULINA 100 UI/ML (ULTRARRAPIDA) 3ML DISPOSITIVO MULTIDOSIS',
 'METFORMINA 500 MG COMPRIMIDOS DE LIBERACIÓN CONVENCIONAL. ENVASE 500 COMPRIMIDOS',
 'DAPAGLIFOZINA METFORMINA 10/1000 X 56 TABLETAS SIMILAR A XIGDUO',
 'JERINGA INSULINA 1CC 29 GX 1/2\tUNIDAD']

In [ ]:
!pip install googletrans==3.1.0a0
from googletrans import Translator
translator = Translator(service_urls=['translate.googleapis.com'])
example = CHLDiab1DirtyNameU[0]
translation = translator.translate(example, dest='en')
print(translation.text)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
GLICENEX SR 1000 METFORMINA 1000 MG


In [ ]:
CHLDiab1DirtyNameU[0]

'GLICENEX SR 1000 METFORMINA 1000 MG'

In [ ]:
# get the orignila index about medicine name for each drug name 

from nltk.tokenize import WordPunctTokenizer  # this function split the string by space
tokenizer = WordPunctTokenizer()

CHLDiab1DirtyNameUSpl = []
for item in CHLDiab1DirtyNameU:
  tokenlist = tokenizer.tokenize(item)
  CHLDiab1DirtyNameUSpl.append(tokenlist)

In [ ]:
CHLDiab1DirtyNameUSpl[:1]

[['GLICENEX', 'SR', '1000', 'METFORMINA', '1000', 'MG']]

In [ ]:
CHLDiab1DirtyNameIndex = []
for item in CHLDiab1DirtyNameUSpl:
  templist = []
  for subitem in item:
    if len(subitem) > 4 and subitem.isdigit() is False:  ## ignoring these elements such as "MG", "+", ... 
      templist.append(subitem)
  CHLDiab1DirtyNameIndex.append(templist)

In [ ]:
CHLDiab1DirtyNameIndex[:1]

[['GLICENEX', 'METFORMINA']]

In [ ]:
# translate these names (indexes) to English
CHLDiab1DirtyNameIndexEng = []

for item in CHLDiab1DirtyNameIndex:
  temp_sublist = []
  for subitem in item:
    temp = translator.translate(subitem, dest='en')
    temp_sublist.append(temp.text)
  CHLDiab1DirtyNameIndexEng.append(temp_sublist)

# this block runs 21 mins

In [ ]:
# delete the information in each piece of drug name after SIMILAR
def DelSIMILAR(subitem):
  finalindex = len(subitem)
  for subindex in range(0, len(subitem)):
    if subitem[subindex] == 'SIMILAR':
      finalindex = subindex
  return subitem[:finalindex]

In [ ]:
# delete the information in each piece of drug name after EQUIVALENT
def DelEQUIVALENT(subitem):
  finalindex = len(subitem)
  for subindex in range(0, len(subitem)):
    if subitem[subindex] == 'EQUIVALENT':
      finalindex = subindex
  return subitem[:finalindex]

In [ ]:
CHLDiab1DirtyNameIndexEngDelSIMILAR = []
for item in CHLDiab1DirtyNameIndexEng:
  add = DelSIMILAR(item)
  add = DelEQUIVALENT(add)
  CHLDiab1DirtyNameIndexEngDelSIMILAR.append(add)

In [ ]:
# for the vocabulaies which are included in the medicine index get from clean data set, keep them in the 
# Chile 1 data index
def KeepMetAnd(subitem):
  addlist = []
  for element in subitem:
    if element in CatalogMedName:
      addlist.append(element)
  return addlist

In [ ]:
CHIfinIndex = []
for sublist in CHLDiab1DirtyNameIndexEngDelSIMILAR:
  add = KeepMetAnd(sublist)
  CHIfinIndex.append(add)

In [ ]:
# count the number of Chile index sub list which is null []
count = 0
for i in CHIfinIndex:
  if not i:
    count += 1

In [ ]:
count

447

In [ ]:
nullindexlist = []
for i in range(0, len(CHIfinIndex)):
  if not CHIfinIndex[i]:
    nullindexlist.append(i)

In [ ]:
!pip install -U spacy
!pip install scispacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached spacy-3.4.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.3 MB)
     |████████████████████████████████| 821 kB 4.3 MB/s 
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.17
    Uninstalling thinc-8.0.17:
      Successfully uninstalled thinc-8.0.17
  Attempting uninstall: spacy
    Found existing installation: spacy 3.0.8
    Uninstalling spacy-3.0.8:
      Successfully uninstalled spacy-3.0.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scispacy 0.5.0 requires spacy<3.3.0,>=3.2.0, but you have spacy 3.4.1 which is incompatible.
en-ner-jnlpba-md 0.4.0 requires spacy<3.1.0,>=3.0.1, but you have spacy 3.4.1 which is incompatible.
en-ner-craft-md 0.4.0 requires spacy<3.1.0,>=3.0.1, but you have spacy 3.4.1 which

In [ ]:
## Install scispaCy models
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_sm-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_craft_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_jnlpba_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bc5cdr_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bionlp13cg_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_lg-0.4.0.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_sm-0.4.0.tar.gz (15.6 MB)
  Using cached spacy-3.0.8-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.8 MB)
  Attempting uninstall: spacy
    Found existing installation: spacy 3.2.4
    Uninstalling spacy-3.2.4:
      Successfully uninstalled spacy-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scispacy 0.5.0 requires spacy<3.3.0,>=3.2.0, but you have spacy 3.0.8 which is incompatible.
en-core-web-sm 3.4.0 requires spacy<3.5.0,>=3.4.0, but you have spacy 3.0.8 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/release

In [ ]:
import scispacy
import spacy

#Core models
import en_core_sci_sm
import en_core_sci_lg

#NER specific models
import en_ner_craft_md
import en_ner_bc5cdr_md
import en_ner_jnlpba_md
import en_ner_bionlp13cg_md

#Tools for extracting & displaying data
from spacy import displacy
import pandas as pd

In [ ]:
nullindexlist[:10]

[11, 14, 16, 28, 30, 53, 57, 58, 96, 100]

In [ ]:
CHLDiab1DirtyNameIndexEngDelSIMILAR[11]

['HIPOGLUCIN', '1000MG']

In [ ]:
#Load specific model and pass text through
nlp = en_core_sci_lg.load()

In [ ]:
#Load specific model and pass text through
#nlp = en_core_sci_lg.load()
#doc = nlp(CHLDiab1DirtyNameIndexEngDelSIMILAR3)

#Display resulting entity extraction
#displacy_image = displacy.render(doc, jupyter=True,style='ent')

an example for spacy

In [ ]:
CHLDiab1DirtyNameIndexEngDelSIMILAR[11]

['HIPOGLUCIN', '1000MG']

In [ ]:
CHLDiab1DirtyNameIndexEngDelSIMILAR11 = str(' '.join(CHLDiab1DirtyNameIndexEngDelSIMILAR[11]))

In [ ]:
doc = nlp(CHLDiab1DirtyNameIndexEngDelSIMILAR11)

#Display resulting entity extraction
displacy_image = displacy.render(doc, jupyter=True,style='ent')

In [ ]:
doc.text

'HIPOGLUCIN 1000MG'

for these drug names in Chile 1, which are not paired with clean index, use spacy nlp to choose the appropriate name

In [ ]:
Chile1drugOutDataName = []
for nullindex in nullindexlist:
  tempChiledrugname = CHLDiab1DirtyNameIndexEngDelSIMILAR[nullindex]
  tempChiledrugname_str = str(' '.join(tempChiledrugname))
  doc = nlp(tempChiledrugname_str)
  temptext = doc.text
  Chile1drugOutDataName.append(temptext)

In [ ]:
Chile1drugOutDataName[:10]

['HIPOGLUCIN 1000MG',
 'Decades',
 'Decades',
 'TABLETS Jalra',
 'X IG duo CONTAINER TABLETS',
 'GLICENEX',
 'GLUCOVANCE',
 'GLAFORNIL',
 'HIPOGLUCIN TABLETS',
 'SYRINGE ACRYLIC 222b 1098']

there are still 12 drug names without index

In [ ]:
Chile1drugOutDataNameNullNumber = 0

for i in Chile1drugOutDataName:
  if not i:
    Chile1drugOutDataNameNullNumber += 1

In [ ]:
Chile1drugOutDataNameNullNumber

12

In [ ]:
Chile1drugOutDataNameNullIndex = []

for i in range(0, len(Chile1drugOutDataName)):
  if not Chile1drugOutDataName[i]:
    Chile1drugOutDataNameNullIndex.append(i)


In [ ]:
Chile1drugOutDataNameNullIndex

[31, 36, 41, 56, 95, 116, 122, 158, 336, 347, 381, 418]

In [ ]:
# for these null element in CHIfnIndex list, use Chile1drugOutDataName to complete them
j = 0
for i in nullindexlist:
  CHIfinIndex[i] = Chile1drugOutDataName[j]
  j += 1


In [ ]:
len(nullindexlist)

447

In [ ]:
for i in range(0, len(CHIfinIndex)):
  if not CHIfinIndex[i]:
    print(i)

350
404
455
590
906
1056
1096
1375
2743
2823
3121
3423


In [ ]:
for index in range(len(CHIfinIndex)):
  if not CHIfinIndex[index]:
    CHIfinIndex[index] = CHLDiab1DirtyNameIndexEngDelSIMILAR[index]

In [ ]:
nullindex = []
for index in range(len(CHIfinIndex)):
  if not CHIfinIndex[index]:
    nullindex.append(index)

In [ ]:
nullindex

[350, 404, 455, 590, 906, 1056, 1096, 1375, 2743, 2823, 3121, 3423]

## Part 3: compare one durg name with each index and select one index to pair

### part 3 - approach 1 (according to the same vocabularies)

In [ ]:
# try to pair the 

myfirstlist = []

for item in CHLDiab1DirtyNameIndexEng[0]:
  for index in MainMedicineIndex:
    add = False
    for subindex in index:
      if subindex == item:
        add = True
    if add == True:
      myfirstlist.append(index)


In [ ]:
CHLDiab1DirtyNameIndexEng[0]

['INSULIN',
 'GLULISINA',
 'CARTRIDGE',
 'DEVICE',
 'PRELLENADO',
 'DISPOSABLE',
 'DISPENSATION']

In [ ]:
myfirstlist

[['INSULIN', 'GLARGINE', 'LIXISENATIDE'],
 ['INSULIN', 'ASPART'],
 ['INSULIN', 'LISPRO'],
 ['INSULIN', 'DEGLUDEC', 'LIIRAGLUTIDE'],
 ['INSULIN', 'GLARGINE'],
 ['INSULIN', 'HUMAN'],
 ['INSULIN', 'SOLUBLE'],
 ['INSULIN', 'GLARGINE'],
 ['INSULIN', 'ASPART'],
 ['INSULIN', 'GLARGINE', 'LIXISENATIDE'],
 ['INSULIN', 'ASPART'],
 ['INSULIN', 'HUMAN'],
 ['INSULIN', 'GLULISINE'],
 ['INSULIN', 'GLULISINE'],
 ['INTERMEDIATE', 'ACTING', 'INSULIN'],
 ['INSULIN'],
 ['INSULIN'],
 ['INSULIN', 'DEGLUDEC'],
 ['INSULIN', 'HUMAN'],
 ['INSULIN', 'HUMAN'],
 ['INSULIN', 'GLARGINE'],
 ['INSULIN', 'DETEMIR'],
 ['INSULIN', 'LISPRO']]

### part 3 - approach 2 (using simhash calculate distance)

In [ ]:
# try to pair the againnnnnn

!pip install simhash

from simhash import Simhash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pairIndexlist = []
for dirtyname in CHIfinIndex:
  distance_list = []
  for index in MainMedicineIndex:
    distance = Simhash(index).distance(Simhash(dirtyname))
    distance_list.append(distance)
  index_min = min(range(len(distance_list)), key=distance_list.__getitem__) # get the index which correspond to the minimum distance
  add_index = MainMedicineIndex[index_min]
  pairIndexlist.append(add_index)

In [ ]:
PairIndexData = {"Chile 1 drug names": CHIfinIndex, "Catalog Index": pairIndexlist}

In [ ]:
print(len(CHIfinIndex), len(pairIndexlist))

3663 3663


In [ ]:
PairIndexData_df = pd.DataFrame(PairIndexData)

In [ ]:
PairIndexData_df

,Chile 1 drug names,Catalog Index
0,[METFORMIN],[METFORMIN]
1,[INSULIN],[INSULIN]
2,[METFORMIN],[METFORMIN]
3,[METFORMIN],[METFORMIN]
4,[INSULIN],[INSULIN]
...,...,...
3658,"[INSULIN, RECOMBINANT]","[INSULIN, DETEMIR]"
3659,"[METFORMIN, HYDROCHLORIDE]","[METFORMIN, HYDROCHLORIDE]"
3660,GALVUS,[FLUCONAZOLE]
3661,THRESIBA FLEXTOUCH,"[EMPAGLIFLOZIN, METFORMIN]"


## Part 4: combine the Chile drug name index pair data frame and full catagol name index data frame to get the result

In [ ]:
# for Chile pair index dataframe, transform list to string
str_pairIndexlist = []
for item in pairIndexlist:
  newitem = ' '.join(item)
  str_pairIndexlist.append(newitem)

In [ ]:
str_CHIfinIndex = []
for item in CHIfinIndex:
  newitem = ' '.join(item)
  str_CHIfinIndex.append(newitem)

In [ ]:
newPairIndexData = {"Chile 1 drug names": str_CHIfinIndex, "Catalog Index": str_pairIndexlist}

In [ ]:
newPairIndexData_df = pd.DataFrame(newPairIndexData)

In [ ]:
newPairIndexData_df[:2]

,Chile 1 drug names,Catalog Index
0,METFORMIN,METFORMIN
1,INSULIN,INSULIN


In [ ]:
# for catalog name index dataframe, transform list to string
str_MainMedicineIndex = []
for item in MainMedicineIndex:
  newitem = ' '.join(item)
  str_MainMedicineIndex.append(newitem)

In [ ]:
newCatalogIndexData = {"Catalog Name": CatalogNameU, "Catalog Index": str_MainMedicineIndex}

In [ ]:
newCatalogIndexData_df = pd.DataFrame(newCatalogIndexData)

In [ ]:
newCatalogIndexData_df[:2]

,Catalog Name,Catalog Index
0,"PHENOBARBITAL 50 MG, TABLET, TBD",PHENOBARBITAL
1,"CARVEDILOL 6.25 MG, TABLET, TBD",CARVEDILOL


In [ ]:
combine_pair_df = pd.merge(newPairIndexData_df, newCatalogIndexData_df, on='Catalog Index', how='left')

In [ ]:
combine_pair_df.to_excel("combine_pair_df_chile_1.xlsx", sheet_name = 'combine_pair_df chile 1')  

## Part 5: Combine the original drug name in Chile 1 with the pairing data frame we have got, then select for the catalog name second time (compute the distance between the original drug name and the sub set of catalog under each name in our result)

In [ ]:
CHLDiab1DirtyName[:5]

['Glicenex SR 1000 Metformina 1000 MG',
 'INSULINA 100 UI/ML (ULTRARRAPIDA) 3ML DISPOSITIVO MULTIDOSIS',
 'Metformina 500 Mg Comprimidos De Liberación Convencional. Envase 500 Comprimidos',
 'dapaglifozina metformina 10/1000 x 56 tabletas similar a xigduo',
 'JERINGA INSULINA 1CC 29 GX 1/2\tUnidad']

In [ ]:
str_CHLDiab1DirtyNameIndexEng[:5]

['GLICENEX METFORMIN',
 'INSULIN ULTRA FAST DEVICE MULTIDOSIS',
 'METFORMIN TABLETS RELEASE CONVENTIONAL CONTAINER TABLETS',
 'DAPAGLIFOZINA METFORMIN TABLETS SIMILAR X IG duo',
 'SYRINGE INSULIN UNIT']

In [ ]:
print(len(CHLDiab1DirtyName), len(str_CHIfinIndex))

3663 3663


In [ ]:
# translate the original full drug name into english

CHLDiab1DirtyNameEng = []

for item in CHLDiab1DirtyName:
  temp = translator.translate(item, dest='en')
  CHLDiab1DirtyNameEng.append(temp.text)

# this block runs 7 mins

In [ ]:
# uppercase everything
CHLDiab1DirtyNameEngU = []
for i in CHLDiab1DirtyNameEng:
  CHLDiab1DirtyNameEngU.append(i.upper())

CHLDiab1DirtyNameEngU[:5]

['GLICENEX SR 1000 METFORMINA 1000 MG',
 'INSULIN 100 IU/ML (ULTRARAPID) 3ML MULTIDOSE DEVICE',
 'METFORMIN 500 MG CONVENTIONAL RELEASE TABLETS. CONTAINER 500 TABLETS',
 'DAPAGLIFOZIN METFORMIN 10/1000 X 56 TABLETS SIMILAR TO XIGDUO',
 'INSULIN SYRINGE 1CC 29 GX 1/2 UNIT']

In [ ]:
# combine uppercase english chile 1 drug name with its indexes
Chile1drugfn_index = {"Chile drugs english uppercase names": CHLDiab1DirtyNameEngU, "Chile 1 drug names": str_CHIfinIndex}

In [ ]:
Chile1drugfn_index_df = pd.DataFrame(Chile1drugfn_index)

In [ ]:
Chile1drugfn_index_df[:2]

,Chile drugs english uppercase names,Chile 1 drug names
0,GLICENEX SR 1000 METFORMINA 1000 MG,METFORMIN
1,INSULIN 100 IU/ML (ULTRARAPID) 3ML MULTIDOSE D...,INSULIN


In [ ]:
# combine Chile drug original full name, its index with the result data frame (pair with catalog name)

Chile1drugs = pd.merge(Chile1drugfn_index_df, combine_pair_df, on='Chile 1 drug names', how='left')

In [ ]:
Chile1drugs[:5]

,Chile drugs english uppercase names,Chile 1 drug names,Catalog Index,Catalog Name
0,GLICENEX SR 1000 METFORMINA 1000 MG,METFORMIN,METFORMIN,"METFORMIN 1000 MG, TABLET, TBD"
1,GLICENEX SR 1000 METFORMINA 1000 MG,METFORMIN,METFORMIN,"METFORMIN 750 MG, TABLET, TBD"
2,GLICENEX SR 1000 METFORMINA 1000 MG,METFORMIN,METFORMIN,"METFORMIN 850 MG, TABLET, TBD"
3,GLICENEX SR 1000 METFORMINA 1000 MG,METFORMIN,METFORMIN,"METFORMIN 500 MG, TABLET, TBD"
4,GLICENEX SR 1000 METFORMINA 1000 MG,METFORMIN,METFORMIN,"METFORMIN 1000 MG, TABLET, TBD"


In [ ]:
print(len(Chile1drugs), len(set(Chile1drugs['Chile drugs english uppercase names'])))

9941941 3404


In [ ]:
# get the list that contains first pairing names list for each original drug full name
fistpairinglist = []
i = 0
for name in CHLDiab1DirtyNameEngU:
  subpairlist = []
  while i < len(Chile1drugs):
    if Chile1drugs['Chile drugs english uppercase names'][i] == name:
      subpairlist.append(Chile1drugs['Catalog Name'][i])
      i += 1
    else:
      break
  fistpairinglist.append(subpairlist)
# runs 4 mins

In [ ]:
print(len(fistpairinglist), len(CHLDiab1DirtyNameEngU))

3663 3663


In [ ]:
finalpairname = []
for item in range(len(CHLDiab1DirtyNameEngU)):
  distance_list = []
  for pairname in fistpairinglist[item]:
    distance = Simhash(CHLDiab1DirtyNameEngU[item]).distance(Simhash(pairname))
    distance_list.append(distance)
  index_min = min(range(len(distance_list)), key=distance_list.__getitem__)
  add_index = fistpairinglist[item][index_min]
  finalpairname.append(add_index)

In [ ]:
# combine the final paired name with the Chile 1 drug english name
Chile1drugfn_index = {"Chile drugs english uppercase names": CHLDiab1DirtyNameEngU, "Final paired catalog name": finalpairname}

In [ ]:
Chile1drugfn_index_df = pd.DataFrame(Chile1drugfn_index)

In [ ]:
Chile1drugfn_index_df[:5]

,Chile drugs english uppercase names,Final paired catalog name
0,GLICENEX SR 1000 METFORMINA 1000 MG,"METFORMIN 1000 MG, TABLET, TBD"
1,INSULIN 100 IU/ML (ULTRARAPID) 3ML MULTIDOSE D...,"INSULIN NPH 100 UI/ML, SOLUTION FOR INJECTION,..."
2,METFORMIN 500 MG CONVENTIONAL RELEASE TABLETS....,"METFORMIN 850 MG, TABLET, TBD"
3,DAPAGLIFOZIN METFORMIN 10/1000 X 56 TABLETS SI...,"METFORMIN 1000 MG, TABLET, TBD"
4,INSULIN SYRINGE 1CC 29 GX 1/2 UNIT,"INSULIN NPH 100 UI/ML, SOLUTION FOR INJECTION,..."


In [ ]:
Chile1drugfn_index_df.to_excel("Chile1drugfn_index.xlsx", sheet_name = 'Chile1drugfn_index chile 1')  